**Theoretical Basis: Shapley Values**
To interpret the "Black Box" of my Neural Network, I utilize **SHAP (SHapley Additive exPlanations)** (Lundberg & Lee, 2017). This method, rooted in Cooperative Game Theory, assigns a contribution value to each feature (token) towards the final prediction. It allows me to verify if my model is learning genuine semantic distinctions or merely overfitting to spurious correlations (the "Clever Hans" effect).

*   **Reference:** Lundberg, S. M., & Lee, S. I. (2017). "A Unified Approach to Interpreting Model Predictions." *NeurIPS*.


In [ ]:
%pip install shap transformers peft torch pandas numpy scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.0/556.0 kB 4.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [shap]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import torch
import shap
import scipy as sp
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from sklearn.model_selection import train_test_split
from datasets import Dataset

df_all = pd.read_csv("precog_task0_data.csv")
df_all['label'] = df_all['class'].apply(lambda x: 0 if x == 'Human' else 1)
print("Data loaded.")


Data loaded.


### Step 1: Restore the Tier C Model
To analyze the model, I need it in memory. I will quickly reinstantiate and train the model on the data to ensure I have the exact object ready for SHAP analysis. (This is fast due to the small dataset size).

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

data = pd.DataFrame({'text': df_all['text'].astype(str), 'label': df_all['label']})
train_df, test_df = train_test_split(data, test_size=0.2, stratify=data['label'], random_state=42)

hf_train = Dataset.from_pandas(train_df)
hf_test = Dataset.from_pandas(test_df)

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=128)

tokenized_train = hf_train.map(preprocess_function, batched=True)
tokenized_test = hf_test.map(preprocess_function, batched=True)

base_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1,
    target_modules=["q_lin", "v_lin"]
)
model = get_peft_model(base_model, peft_config)

class_counts = train_df['label'].value_counts()
class_weights = torch.tensor([len(train_df) / (2 * class_counts[0]), 
                              len(train_df) / (2 * class_counts[1])], dtype=torch.float32)

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

training_args = TrainingArguments(
    output_dir="./results_explain", learning_rate=2e-4, per_device_train_batch_size=8, 
    num_train_epochs=3, weight_decay=0.01, logging_steps=10
)

trainer = WeightedTrainer(model=model, args=training_args, train_dataset=tokenized_train, tokenizer=tokenizer)
trainer.train()


Map:   0%|          | 0/796 [00:00<?, ? examples/s]

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/xx/t73hkxhx7xj95d9rl4spgtb00000gn/T/ipykernel_8477/1013259671.py:46: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(model=model, args=training_args, train_dataset=tokenized_train, tokenizer=tokenizer)
/opt/homebrew/lib/python3.11/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,0.536400
20,0.273500
30,0.088300
40,0.025400
50,0.019100
60,0.018800
70,0.003100
80,0.000300
90,0.011900
100,0.000300


TrainOutput(global_step=300, training_loss=0.03276905312978973, metrics={'train_runtime': 40.6588, 'train_samples_per_second': 58.733, 'train_steps_per_second': 7.378, 'total_flos': 80439425888256.0, 'train_loss': 0.03276905312978973, 'epoch': 3.0})

### Step 2: Saliency Mapping with SHAP
I will now examine which words contribute most to the "AI" classification.

In [ ]:
def f(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=128, truncation=True) for v in x]).to(model.device)
    attention_mask = (tv != 0).type(torch.int64).to(model.device)
    outputs = model(tv, attention_mask=attention_mask)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(axis=1)).T
    return scores

explainer = shap.Explainer(f, tokenizer)

ai_samples = df_all[df_all['label'] == 1]['text'].head(5).tolist()
text_to_explain = ai_samples[0]

print("Analyzing AI Sample :")
print(text_to_explain[:200])

shap_values = explainer([text_to_explain])

shap.plots.text(shap_values)


Analyzing AI Sample:
The tension between the singular human spirit and the collective structure of society remains one of the most enduring themes in the literary and philosophical canon. At its core, this conflict arises...


### Step 3: Error & Edge Case Analysis
I check if any Human texts were misclassified. If the accuracy is 100%, I look for the "least confident" correct predictions (Edge Cases).

In [ ]:
def get_ai_probability(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(model.device)
    with torch.no_grad():
        logits = model(**inputs).logits
    probs = torch.softmax(logits, dim=1)
    return probs[0][1].item()

human_df = df_all[df_all['label'] == 0].copy()
human_df['ai_prob'] = human_df['text'].apply(get_ai_probability)

errors = human_df[human_df['ai_prob'] > 0.5]

print(f"Number of False Positives: {len(errors)}")

if len(errors) > 0:
    print("\n--- Analysis of Errors ---")
    for i, row in errors.head(3).iterrows():
        print(f"\nMisclassified Sample {i} (Prob AI: {row['ai_prob']:.4f}):")
        print(row['text'])
else:
    print("\nNo direct misclassifications found.")
    print("Showing 'Edge Cases' - Human samples closest to AI boundary:")
    edge_cases = human_df.sort_values('ai_prob', ascending=False).head(3)
    for i, row in edge_cases.iterrows():
        print(f"\nEdge Case (Prob AI: {row['ai_prob']:.4f}):")
        print(row['text'])


Number of False Positives (Human labeled as AI): 0

No direct misclassifications found (Model is 100% accurate).
Showing 'Edge Cases' - Human samples with highest AI probability:

Edge Case (Prob AI: 0.0664):
It is not seldom the case that when a man is browbeaten in some unprecedented and violently unreasonable way, he begins to stagger in his own plainest faith. He begins, as it were, vaguely to surmise that, wonderful as it may be, all the justice and all the reason is on the other side. Accordingly, if any disinterested persons are present, he turns to them for some reinforcement for his own faltering mind.

Edge Case (Prob AI: 0.0404):
“A man who has once been refused! How could I ever be foolish enough to expect a renewal of his love? Is there one among the sex who would not protest against such a weakness as a second proposal to the same woman? There is no indignity so abhorrent to their feelings.”

Edge Case (Prob AI: 0.0215):
“Pride,” observed Mary, who piqued herself upon the

### Interpretation

My SHAP analysis confirms the hypothesis from Task 2.
*   **Observation:** The model places extreme weight on function words and stylistic markers common in the 1800s but rare today.
*   **The Verdict:** The model is effectively learned that *Humans = 1850s* and *AI = 2026*.
*   **Theoretical Concept:** This is known as **Spurious Correlation** or the **Clever Hans Effect** in ML. The model gets the right answer (100% accuracy) for the wrong reason (detecting time period, not humanity).
*   **Actionable Insight:** To build a *real* AI detector, I would need to control for specific domains (e.g., train on Modern Human vs. Modern AI, and Antique Human vs. Antique AI) to force the model to learn subtle statistical artifacts rather than obvious vocabulary shifts.
